In [1]:
import requests
from pathlib import Path
import json
import pandas as pd
import datetime
from datetime import datetime, timezone
import hashlib
from sqlalchemy import create_engine
import numpy as np
from time import sleep
import logging
import pytz
import math
import os

In [2]:
from dotenv import load_dotenv
env_file = Path("../.env")
print(".env exists:", env_file.exists(), "→", env_file)
# Загружаем .env
loaded = load_dotenv(dotenv_path=env_file)

.env exists: True → ..\.env


In [3]:
pd.set_option('display.max_columns', None)  # Отображать все столбцы
pd.set_option('display.width', 1000)  # Увеличить ширину отображения

In [4]:
# Текущее время
current_time = datetime.now(timezone.utc).isoformat()
current_time

'2025-12-08T11:26:27.363402+00:00'

In [6]:
url = os.getenv('URL')
headers = {
    'Content-Type': os.getenv('CONTENT_TYPE'),
    'Authorization': os.getenv('AUTHORIZATION'),
}
REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')

**Обновление пары Access Token и Refresh Token**

In [ ]:
url_ref = "https://demo-hs.fgis-saturn.ru/backend/role/newaccesstoken"

payload = {}
headers = {
  'refresh_token': REFRESH_TOKEN 
}

# Отправляя запрос, нужно понимать что пару токенов прийдется обновить

#response = requests.request("GET", url_ref, headers=headers, data=payload)

#print(response.text)

**Проверка регистрации компании в ФГИС САТУРН**

In [ ]:
payload = json.dumps({
  "com": "execOperation",
  "op": "static/getList()",
  "otype": "Contractor",
  "opargs":{
    "pos": 0,
    "size": 10,
    "getFullCards": 1,
    "filters": [
      {
        "column": "inn",
        "condition": "=",
        "value": [5029069967] # ИНН проверяемой компании
      },
      { "column": "lcState",
       "condition": "=", 
       "value": ["actual"] 
      }
    ]
  }
})

response = requests.post(url, headers=headers, data=payload)

#print(response.status_code)

registr_data = response.json()

In [ ]:
# Пример ответа от САТУРН в формате HRJSON
registr_data

{'transId': None,
 'resCode': 200,
 'reqPerMin': None,
 'resMsg': None,
 'resDescription': None,
 'resData': {'objList': {'_FORMAT_VER': '2021.09.25_0430',
   '_rectype': 'array',
   '_OBJ_ARRAY': [{'id': '248824',
     'name': 'ООО "ЛЕ МОНЛИД"',
     'INN': '5029069967',
     'lcState': 'actual',
     'personFio': None,
     'legalAddress': '141031, Российская Федерация, Московская обл., г. Мытищи, Осташковское ш., д. 1',
     'factualAddress': '',
     'vetis_region': None,
     'responsiblePerson': 'ДЕФАССЬЕ ЛОРАН, ЛУИ, КЛОД ;ГЕНЕРАЛЬНЫЙ ДИРЕКТОР',
     'phoneNumber': '',
     'email': '',
     'KPP': '502901001',
     'OGRN': '1035005516105',
     'dateRegistration': 1055894400.0,
     'dateModified': '2025-03-16T23:01:17.564097458Z',
     'vetis_versionUUID': 'd6638f09-c355-45d1-9e13-f4dec68f774c',
     'vetis_type': 1,
     'vetis_incorporationForm': 'Имя: Общества с ограниченной ответственностью Код: 12300 Короткое имя: null',
     'fullName': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕН

In [ ]:
# Пример работы с HRJSON как с обычным JSON

# Извлекаем нужные массивы
obj_array = registr_data["resData"]["objList"]["_OBJ_ARRAY"]

# Проверяем, что есть хотя бы один объект в массиве
if obj_array:
    obj = obj_array[0]
    
    # Выводим необходимые поля
    print("ID компании:", obj["id"])
    print("Наименование компании:", obj["name"])
    print("ИНН:", obj["INN"])
    print("Статус:", obj["lcState"])
    print("Руководитель:", obj["responsiblePerson"])
    print("Юридический адрес:", obj["legalAddress"])
else:
    print("Компания не зарегестрирована во ФГИС САТУРН")

ID компании: 248824
Наименование компании: ООО "ЛЕ МОНЛИД"
ИНН: 5029069967
Статус: actual
Руководитель: ДЕФАССЬЕ ЛОРАН, ЛУИ, КЛОД ;ГЕНЕРАЛЬНЫЙ ДИРЕКТОР
Юридический адрес: 141031, Российская Федерация, Московская обл., г. Мытищи, Осташковское ш., д. 1
